In [1]:
import keras

Using TensorFlow backend.


In [2]:

import numpy as np
import pandas as pd
from sklearn.model_selection import KFold
from sklearn import metrics
import keras
from keras.models import Model ,Sequential
from keras.optimizers import Adam
from keras.applications.inception_v3 import InceptionV3
from keras.layers import Dense, Input, Flatten, Dropout, GlobalAveragePooling2D ,Conv2D ,Activation , Input
from keras.layers.normalization import BatchNormalization
from keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau

In [3]:
import keras.backend as K
K.clear_session()
cfg = K.tf.ConfigProto()
cfg.gpu_options.allow_growth = True
K.set_session(K.tf.Session(config=cfg))

In [4]:
import PIL
import cv2
import numpy as np
from matplotlib import pyplot as plt
from PIL.ImageOps import equalize as eq
import random
random.uniform(0.01, 1)

0.8846486939797218

In [5]:
import numpy as np
import random
from PIL.ImageOps import equalize as eq

def white (img):
    img=image.array_to_img (img)
    img= eq(img)
    return image.img_to_array (img)
    

In [6]:
def lap (img):
    img=image.array_to_img (img).convert('RGB') 
 
    open_cv_image = np.array(img) 
# Convert RGB to BGR 
    open_cv_image = open_cv_image[:, :, ::-1].copy() 
    laplacian = cv2.Laplacian(open_cv_image,cv2.CV_64F)
    
    return laplacian

In [7]:
def aug (img):
    x=lap (img)
    
    return white(x)

In [13]:
img_height = 150
img_width = 300
img_channels =3
img_dim = (img_height, img_width, img_channels)

def inceptionv3(img_dim=img_dim):
    input_tensor = Input(shape=img_dim)
    base_model = InceptionV3(include_top=False,
                   weights='imagenet',
                   input_shape=img_dim)
    bn = BatchNormalization()(input_tensor)
    x = base_model(bn)
    x = GlobalAveragePooling2D()(x)
    x = Dropout(0.5)(x)
    output = Dense(4, activation='softmax')(x)
    model = Model(input_tensor, output)
    return model

model = inceptionv3()
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         (None, 150, 300, 3)       0         
_________________________________________________________________
batch_normalization_190 (Bat (None, 150, 300, 3)       12        
_________________________________________________________________
inception_v3 (Model)         (None, 3, 8, 2048)        21802784  
_________________________________________________________________
global_average_pooling2d_2 ( (None, 2048)              0         
_________________________________________________________________
dropout_2 (Dropout)          (None, 2048)              0         
_________________________________________________________________
dense_2 (Dense)              (None, 4)                 8196      
Total params: 21,810,992
Trainable params: 21,776,554
Non-trainable params: 34,438
___________________________________________________________

In [14]:
from keras.preprocessing.image import ImageDataGenerator
from keras.preprocessing import image
gen=image.ImageDataGenerator(rotation_range=30,shear_range=0.2, preprocessing_function=lap )
gen1=image.ImageDataGenerator(preprocessing_function=lap)


In [15]:
batch_size=20

train= gen.flow_from_directory('train', target_size=(150,300),
                class_mode='categorical', shuffle=True, batch_size=batch_size)

Found 5820 images belonging to 4 classes.


In [16]:
val= gen1.flow_from_directory('val', target_size=(150,300),
                class_mode='categorical', shuffle=False, batch_size=batch_size)

Found 1098 images belonging to 4 classes.


In [17]:
model.compile(optimizer=Adam(lr=1e-4), loss='categorical_crossentropy', 
                      metrics = ['accuracy'])

In [18]:
batch_size=20
model.fit_generator(train, steps_per_epoch=5820/batch_size, epochs=3, 
                            validation_data=val, validation_steps=1098/batch_size)

Epoch 1/3
291/291 [==============================] - 173s - loss: 1.0513 - acc: 0.5562 - val_loss: 1.5124 - val_acc: 0.3215
Epoch 2/3
291/291 [==============================] - 172s - loss: 0.7982 - acc: 0.6763 - val_loss: 1.6718 - val_acc: 0.3078
Epoch 3/3
291/291 [==============================] - 176s - loss: 0.6938 - acc: 0.7284 - val_loss: 2.0893 - val_acc: 0.3033


In [ ]:
batch_size=20
model.optimizer.lr=0.1
model.fit_generator(train, steps_per_epoch=5820/batch_size, epochs=3, 
                            validation_data=val, validation_steps=1098/batch_size)

Epoch 1/3
291/291 [==============================] - 207s - loss: 1.0510 - acc: 0.5529 - val_loss: 1.4761 - val_acc: 0.3306
Epoch 2/3
268/291 [==========================>...] - ETA: 14s - loss: 0.9449 - acc: 0.6047

In [34]:
test.classes

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], dtype=int32)

In [47]:
test = gen1.flow_from_directory('newtest', target_size=(150,300),
                class_mode='binary', shuffle=False, batch_size=1 )

Found 115 images belonging to 2 classes.


In [49]:
pre=model.predict_generator(test,115)

In [50]:
files = test.filenames

In [54]:
mistake=0
doubt=0
file=[]
file1=[]
correct=0
num=0
for x in pre :
    
    if x[0]<0.8 and x[0]>0.2: 
        doubt+=1
        print (x , "doubt")
        file1.append (files[num])
    else:
        if num<0:
            if x[0]>0.8: 
                mistake+=1
                print (x,num)
                file.append (files[num])
            else : correct+=1
        else:
            if x[0]<0.8: 
                mistake+=1
                print (x,num) 
                file.append (files[num])
            else : correct+=1
    num+=1
    
all=correct+mistake

[  1.66085073e-07] 0
[ 0.00159416] 1
[ 0.0004852] 2
[ 0.00022313] 3
[  3.69669328e-06] 4
[  4.51046738e-08] 5
[ 0.00041703] 6
[  8.60614382e-05] 7
[  6.33264135e-05] 8
[ 0.00012839] 9
[ 0.00131503] 10
[ 0.00097508] 11
[ 0.00084794] 12
[  1.15112935e-05] 13
[  5.33862476e-05] 14
[ 0.00045947] 15
[  6.98463846e-05] 16
[  2.17293891e-05] 17
[  3.77711367e-05] 18
[  2.36685264e-06] 19
[  2.27249302e-06] 20
[ 0.00051964] 21
[  6.70497830e-05] 22
[ 0.00185021] 23
[  5.28199453e-05] 24
[ 0.00023212] 25
[  2.04589342e-05] 26
[  2.93554021e-05] 27
[ 0.00051169] 28
[  3.21427081e-07] 29
[  3.94702038e-05] 30
[  7.11783941e-05] 31
[  1.29871232e-05] 32
[ 0.00088638] 33
[  4.66361307e-05] 34
[ 0.00021132] 35
[ 0.00128563] 36
[ 0.00456053] 37
[ 0.00010767] 38
[ 0.00027094] 39
[  3.59039109e-06] 40
[ 0.00011883] 41
[  8.23270966e-05] 42
[  2.87157927e-05] 43
[  6.00116482e-06] 44
[ 0.00206607] 45
[ 0.00011634] 46
[ 0.00110863] 47
[  1.97718805e-06] 48
[ 0.00555313] 49
[ 0.00027946] 50
[ 0.00031836] 

In [52]:
correct/all

1.0

In [45]:
doubt/(all+doubt)

0.01680672268907563

In [46]:
mistake

0